# 2_party_continus_input_binay_classification

## 資料品質檢查

### 設定資料路徑 & 參數

In [14]:
import os
guest, host = 9999, None
data_base = "/data/projects/fate/"

if guest:
    dense_data = {"name": "breast_hetero_guest", "namespace": f"experiment"}
    dense_data_dir = os.path.join(data_base, "persistence/data/breast_hetero_guest.csv")
if host:
    dense_data = {"name": "breast_hetero_host", "namespace": f"experiment"}
    dense_data_dir = os.path.join(data_base, "persistence/data/breast_hetero_host.csv")

### 缺失值 & 欄位名

在待會 DataTransform( )中我們可以
1. 填補缺失值
2. 指定標籤欄位名

In [67]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
dense_df.isna().sum()

id        0
target    0
x0        0
x1        0
x2        1
x3        1
x4        2
x5        1
x6        1
x7        0
x8        0
x9        2
dtype: int64

## 上傳資料

In [57]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-05-31 05:48:15.476 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305310548153195420

2023-05-31 05:48:15.484 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-05-31 05:48:16.501 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 05:48:16.502 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-05-31 05:48:17.519 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-05-31 05:48:18.536 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-05-31 05:48:19.552 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-05-31 05:48:20.570 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-05-31 05:48:21.588 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-05-31 

### 建構 Training pipeline 範例

使用 `pipeline` 模塊來構建聯邦學習流程

In [143]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [150]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=10000)

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [145]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

In [146]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True, label_name='target', label_type='int', 
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [147]:
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [148]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [149]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [151]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-05-31 06:31:52.482 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305310631521197550

2023-05-31 06:31:52.492 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-05-31 06:31:53.502 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-05-31 06:31:54.526 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 06:31:54.528 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-05-31 06:31:55.546 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-05-31 06:31:56.572 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-05-31 06:31:57.593 | INFO     | pipel

2023-05-31 06:32:29.763 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-05-31 06:32:30.785 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-05-31 06:32:31.803 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
m2023-05-31 06:32:32.825 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 06:32:32.826 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:00:40
2023-05-31 06:32:33.846 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:00:41
2023-05-31 06:32:34.863 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0,

m2023-05-31 06:33:08.883 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 06:33:08.884 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:16
2023-05-31 06:33:09.904 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-05-31 06:33:10.923 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-05-31 06:33:11.941 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-05-31 06:33:12.963 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-05-31 06:33:13.981 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - 

2023-05-31 06:33:45.937 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:53
2023-05-31 06:33:46.957 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-05-31 06:33:47.993 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-05-31 06:33:49.046 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-05-31 06:33:50.067 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-05-31 06:33:51.086 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-05-31 06:33:52.106 | INFO     | pip

2023-05-31 06:34:23.824 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:31
2023-05-31 06:34:24.843 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:32
2023-05-31 06:34:25.861 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:33
2023-05-31 06:34:26.884 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:34
2023-05-31 06:34:27.904 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:35
2023-05-31 06:34:28.922 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:36
2023-05-31 06:34:29.945 | INFO     | pip

...

2023-05-31 06:38:48.921 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:06:56
2023-05-31 06:38:49.939 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:06:57
2023-05-31 06:38:50.959 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:06:58
2023-05-31 06:38:51.976 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:06:59
2023-05-31 06:38:53.003 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:00
2023-05-31 06:38:54.021 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:01
2023-05-31 06:38:55.040 | INFO     | pip

2023-05-31 06:39:26.777 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:34
2023-05-31 06:39:27.796 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:35
2023-05-31 06:39:28.817 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:36
2023-05-31 06:39:29.841 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:37
2023-05-31 06:39:30.871 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:38
2023-05-31 06:39:31.891 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:39
2023-05-31 06:39:32.974 | INFO     | pip

2023-05-31 06:40:04.708 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:12
2023-05-31 06:40:05.730 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:13
2023-05-31 06:40:06.755 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:14
2023-05-31 06:40:07.778 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:15
2023-05-31 06:40:08.796 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:16
2023-05-31 06:40:09.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:17
2023-05-31 06:40:10.835 | INFO     | pip

2023-05-31 06:40:42.645 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:50
2023-05-31 06:40:43.666 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:51
2023-05-31 06:40:44.684 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:52
2023-05-31 06:40:45.703 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:53
2023-05-31 06:40:46.722 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:54
2023-05-31 06:40:47.743 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:55
2023-05-31 06:40:48.761 | INFO     | pip

2023-05-31 06:41:21.535 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:09:29
2023-05-31 06:41:22.554 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:09:30
2023-05-31 06:41:23.587 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:09:31
2023-05-31 06:41:24.610 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:09:32
2023-05-31 06:41:25.628 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:09:33
2023-05-31 06:41:27.683 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:89 - Job is success!!! Job id is 202305310631521197550
2023-05-31 06:41:27.684 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status

當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [152]:
pipeline.dump("pipeline_saved/pipeline_saved.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `intersect_0`, `hetero_secureboost_0`

In [153]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [154]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [155]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

整合所有模塊

In [157]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [158]:
predict_pipeline.predict()

2023-05-31 06:42:14.075 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202305310642135516890

2023-05-31 06:42:14.084 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-05-31 06:42:15.096 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-05-31 06:42:16.123 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 06:42:16.125 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:02
2023-05-31 06:42:17.146 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:03
2023-05-31 06:42:18.169 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:04
2023-05-31 06:42:19.237 | INFO     | pipel

2023-05-31 06:42:51.242 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-05-31 06:42:52.261 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-05-31 06:42:53.322 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
m2023-05-31 06:42:55.367 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-05-31 06:42:55.368 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:00:41
2023-05-31 06:42:56.386 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:00:42
2023-05-31 06:42:57.447 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0,

2023-05-31 06:43:30.331 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:16
2023-05-31 06:43:31.349 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-05-31 06:43:32.366 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-05-31 06:43:33.385 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-05-31 06:43:34.402 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-05-31 06:43:35.424 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:21
2023-05-31 06:43:36.442 | INFO     | pip

2023-05-31 06:44:08.407 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-05-31 06:44:09.424 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-05-31 06:44:10.443 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-05-31 06:44:11.461 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-05-31 06:44:12.479 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-05-31 06:44:13.498 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-05-31 06:44:14.540 | INFO     | pip

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [175]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/2_party_continus_classification_tutorial.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/2_party_continus_classification_tutorial.json 
 {
    "hetero_secureboost_0": {
        "train": {
            "auc": 0.995580307594736,
            "ks": 0.9669811320754716
        }
    }
}
